In [1]:
!pip install -qqq google-genai

In [2]:
!pip install -qqq opencv-python ultralytics numpy==1.26.4

In [3]:
import google.generativeai as genai

In [4]:
import google.generativeai as genai
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

try:
    GEMINI_API_KEY = user_secrets.get_secret("GEMINI_API_KEY")
    
    if GEMINI_API_KEY is None:
        raise ValueError("GEMINI_API_KEY not found in secrets.")
        
    genai.configure(api_key=GEMINI_API_KEY)
    gemini_model = genai.GenerativeModel('gemini-2.5-flash')
    print("✅ Gemini API configured securely.")
except Exception as e:
    print(f"⚠️ Could not configure Gemini API. Error: {e}")

✅ Gemini API configured securely.


In [5]:
import os
import time
import json
from pathlib import Path
import cv2
import numpy as np
from PIL import Image
from io import BytesIO

# Imports for Gemini API
import google.generativeai as genai

import requests

In [6]:
YOLO_MODEL_PATH = '/kaggle/input/60-epochs/60_epochs.pt'
VIDEO_PATH = '/kaggle/input/tests-for-llm-yolo/gettyimages-1058-12-640_adpp.mp4'

# Fall Detection Thresholds
CONFIDENCE_THRESHOLD = 0.70  # YOLO confidence score to trigger LLM verification
FALL_CLASS_ID = 0            # Based on the training output {0: 'Fall-Detected'}

# LLM Cooldown
COOLDOWN_PERIOD_SECONDS = 10 # Cooldown to prevent spamming the LLM API

In [7]:
LLM_PROMPT = """
You are an AI vision system designed to assist emergency responders. Analyze the provided image and respond ONLY in valid JSON format with the following keys:

- "fall_detected": boolean,
- "context": string (must be a 2 line description of the condition of the person and the surroundings; estimate the cause of fall as well),
- "bleeding_observed": boolean,
- "person_condition": string (one of: "alert", "unresponsive", "injured", "bleeding", "unknown"),
- "confidence": float (a score in percentage (0-100)% how sure are you about your prediction in percentage)

Do not include any other text.
"""
try:
    from ultralytics import YOLO
    fall_model = YOLO(YOLO_MODEL_PATH)
    print(f"✅ YOLOv8 Model loaded from: {YOLO_MODEL_PATH}")
except Exception as e:
    print(f"❌ Failed to load YOLO model: {e}")

✅ YOLOv8 Model loaded from: /kaggle/input/60-epochs/60_epochs.pt


In [8]:
def verify_fall_with_llm(frame, prompt):
    """
    Sends a frame to the Gemini LLM for high-confidence fall verification.
    Includes diagnostic prints for debugging the hang.
    """
    if 'gemini_model' not in globals():
        print("LLM is not configured. Skipping verification.")
        return None

    try:
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        pil_image = Image.fromarray(rgb_frame)

        # The blocking call happens here
        response = gemini_model.generate_content([prompt, pil_image])

        json_text = response.text.strip()

        # Robustly strip markdown fences
        if json_text.startswith('```json'):
            json_text = json_text.strip().replace('```json', '').replace('```', '').strip()

        return json.loads(json_text)

    except json.JSONDecodeError as e:
        print(f"❌ LLM JSON Decode Error: {e}")
        print(f"LLM Raw Response: {response.text}")
        return None
    except Exception as e:
        print(f"❌ An unexpected error occurred during LLM call (Possible API Timeout): {e}")
        return None

In [14]:
def send_telegram_alert(llm_response):
    """
    Formats the LLM's JSON response and sends it as a Telegram message.
    """
    print("   Attempting to send Telegram alert...")
    try:
        # Load credentials securely from secrets
        BOT_TOKEN = user_secrets.get_secret("TELEGRAM_BOT_TOKEN")
        CHAT_ID = user_secrets.get_secret("TELEGRAM_CHAT_ID")

        if not BOT_TOKEN or not CHAT_ID:
            print("❌ Telegram credentials (TELEGRAM_BOT_TOKEN, TELEGRAM_CHAT_ID) not found in secrets.")
            return

        # Format the message nicely from the LLM's response
        status = llm_response.get('person_condition', 'unknown').upper()
        confidence = llm_response.get('confidence', 0.0)
        context = llm_response.get('context', 'No context provided.')
        bleeding = "Yes" if llm_response.get('bleeding_observed', False) else "No"

        message_text = (
            f"🚨 *HIGH CONFIDENCE FALL DETECTED* 🚨\n\n"
            f"*Status:* {status}\n"
            f"*Confidence:* {confidence:.2f}%\n"
            f"*Bleeding Observed:* {bleeding}\n\n"
            f"*Context from LLM:*\n"
            f"{context}"
        )
        
        # Construct the Telegram API URL
        url = f"https://api.telegram.org/bot{BOT_TOKEN}/sendMessage"
        
        # Create the payload
        payload = {
            'chat_id': CHAT_ID,
            'text': message_text,
            'parse_mode': 'Markdown'  # Allows for formatting (like *bold*)
        }

        # Send the request
        response = requests.post(url, data=payload, timeout=5)
        
        if response.status_code == 200:
            print("   ✅ Telegram alert sent successfully.")
        else:
            print(f"   ❌ Failed to send Telegram alert. Status: {response.status_code}, Response: {response.text}")

    except Exception as e:
        print(f"   ❌ An error occurred while sending Telegram alert: {e}")

In [15]:
if not Path(VIDEO_PATH).exists():
    print(f"❌ Video file not found at: {VIDEO_PATH}")
elif 'fall_model' not in globals():
    print("❌ YOLO model is not loaded. Cannot start processing.")
else:
    cap = cv2.VideoCapture(VIDEO_PATH)

    if not cap.isOpened():
        print("❌ Error opening video stream or file.")
    else:
        frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = cap.get(cv2.CAP_PROP_FPS)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        print(f"Video Info: {frame_width}x{frame_height} @ {fps:.2f} FPS, {total_frames} frames.")

        last_llm_call_time = 0
        frame_count = 0

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            current_time = time.time()
            frame_count += 1

            # --- STAGE 1: Real-Time Edge Detection (YOLOv8) ---
            yolo_triggered = False
            results = fall_model(frame, verbose=False)

            if results and results[0].boxes:
                for box in results[0].boxes:
                    if int(box.cls[0]) == FALL_CLASS_ID and float(box.conf[0]) >= CONFIDENCE_THRESHOLD:
                        yolo_triggered = True
                        break

            # If the YOLO confidence threshold is met:
            if yolo_triggered:
                print(f"\n--- Frame {frame_count} ---")
                print(f"⚠️ YOLO Fall Detection > {CONFIDENCE_THRESHOLD*100:.2f}%! Initiating LLM Verification.")

                # --- STAGE 2: Multimodal LLM Verification (Gemini) ---
                if current_time - last_llm_call_time >= COOLDOWN_PERIOD_SECONDS:

                    llm_start_time = time.time()

                    # This print statement shows we've passed the cooldown check and are entering the blocking call
                    print(f"   ⏳ Sending frame to Gemini... (Will block until response is received)")
                    verification_result = verify_fall_with_llm(frame, LLM_PROMPT)

                    llm_duration = time.time() - llm_start_time

                    if verification_result:
                        last_llm_call_time = current_time # Update cooldown timer

                        is_fall = verification_result.get('fall_detected', False)
                        confidence = verification_result.get('confidence', 0.0)
                        condition = verification_result.get('person_condition', 'unknown')
                        context = verification_result.get('context', 'No context provided.')

                        # Corrected Confidence Logic: LLM output (0-100) vs 80.0
                        if is_fall and confidence >= 80.0:
                            print("🚨 HIGH CONFIDENCE FALL ALERT!")
                            print(f"   Status: {condition.upper()} | Confidence: {confidence:.2f}% | LLM Time: {llm_duration:.2f}s")
                            print(f"   Context: {context}")
                            send_telegram_alert(verification_result)
                            print("----------------------------------------------------------------")
                        else:
                            print(f"✅ LLM Verification: Fall not confirmed or confidence low ({confidence:.2f}%).")
                            print(f"   LLM Time: {llm_duration:.2f}s. Context: {context}")

                    else:
                        print(f"LLM call failed or returned unparseable data. Duration: {llm_duration:.2f}s")

                else:
                    cooldown_remaining = COOLDOWN_PERIOD_SECONDS - (current_time - last_llm_call_time)
                    print(f"⏳ Cooldown in effect. Skipping LLM call. Remaining: {cooldown_remaining:.2f}s")

        cap.release()
        print("\nVideo processing complete. 🎬")

Video Info: 768x432 @ 23.98 FPS, 63 frames.

--- Frame 6 ---
⚠️ YOLO Fall Detection > 70.00%! Initiating LLM Verification.
   ⏳ Sending frame to Gemini... (Will block until response is received)
🚨 HIGH CONFIDENCE FALL ALERT!
   Status: UNKNOWN | Confidence: 97.50% | LLM Time: 11.76s
   Context: A person is actively falling from a skateboard while attempting a trick on a stair handrail.
The fall is occurring on a concrete staircase outside a building, likely due to a loss of balance during the skateboarding maneuver.
   Attempting to send Telegram alert...
   ✅ Telegram alert sent successfully.
----------------------------------------------------------------

--- Frame 10 ---
⚠️ YOLO Fall Detection > 70.00%! Initiating LLM Verification.
   ⏳ Sending frame to Gemini... (Will block until response is received)
🚨 HIGH CONFIDENCE FALL ALERT!
   Status: ALERT | Confidence: 90.00% | LLM Time: 9.52s
   Context: A skateboarder is in mid-air, appearing to lose balance while attempting a trick dow